In [ ]:
#=================================================#
#                  VERSION 0.03                   #
#=================================================#

#=================[DEPENDENCIES]==================#


import random #for shuffling the list of breeding pairs

#===================[FUNCTIONS]===================#

#--------------[GENETIC ALGORITHM]----------------#

def selection(G, score_scheme = "min"):
    
    if score_scheme == "min":

        G_normal = [exp(-1 * element[1]) for element in G]
        
    elif score_scheme == "max":
        
        G_normal = [element[1] for element in G]
        
    G_dist   = GeneralDiscreteDistribution(G_normal)
    G_select = []

    while len(G_select) < len(G):
        
        G_select.append(G[G_dist.get_random_element()])

    return G_select

def pair(G, n):

    random.shuffle(G)
    parent_groups = []

    for i in range(len(G)/n):
        parent_group = []
        for j in range(n):
            parent_group.append(G[i + j*len(G)/n][0])
        parent_groups.append(parent_group)

    return parent_groups

def crossover(parent_groups, event_prob):

    event_dist = GeneralDiscreteDistribution([1 - event_prob, event_prob])
    offspring = []
    for parent_group in parent_groups:
        a_parent = parent_group[0]
        b_parent = parent_group[1]
        
        cross_flag = event_dist.get_random_element() 
        
        if cross_flag == 1:
            cross_point = random.choice([i for i in range(1,min(len(a_parent), len(b_parent)))])
            
            a_offspring = [a_parent[i] for i in range(cross_point)] + [b_parent[i] for i in range(cross_point, len(b_parent))]
            b_offspring = [b_parent[i] for i in range(cross_point)] + [a_parent[i] for i in range(cross_point, len(a_parent))]
            
            if isPreperiodic(a_offspring):
                
                a_offspring = a_parent
                
            if isPreperiodic(b_offspring):
                
                b_offspring = b_parent
            
        else:
            
            a_offspring = a_parent
            b_offspring = b_parent
            
        offspring.append(a_offspring)
        offspring.append(b_offspring)
            
    return offspring       
            
def mutation(G, event_prob, mutation_space):
    
    mutation_dist = GeneralDiscreteDistribution([1 - event_prob, event_prob])
    
    for individual in G:
        
        for g in range(len(individual)):
            
            mutate_flag = mutation_dist.get_random_element()
            
            if mutate_flag == 1:
                
                mutation = random.choice(mutation_space)
                
                individual[g] = mutation
                
    return G

def grade(G, gradeFunc):
    
    new_generation = []
    
    for individual in G:
        
        new_generation.append([individual, gradeFunc(individual)])
    
    return new_generation

def geneticAlgorithm(G, parent_count, cross_prob, mutate_prob, mutate_space, gradeFunc, generation_count, score_scheme = "min", display = False):
    
    generation_counter = 0
    
    elite = G[0]
    
    while generation_counter < generation_count:
        
        G = grade(mutation(crossover(pair(selection(G),2), cross_prob), mutate_prob, mutate_space), gradeFunc)
        
        G.sort(key = lambda x: x[1])
            
        if G[0][1] < elite[1]:
            
            elite = G[0]
        
        if display == True:
            
            if score_scheme == "max":
                
                gen_score = max([individual[1] for individual in G])
                
            else:
                
                gen_score = min([individual[1] for individual in G])
            
            print("Generation", str(generation_counter + 1), "has fitness score", str(gen_score))
            print(G[0])
            
        generation_counter += 1
        
    print("Found elite element", str(toPoint(elite[0], L)), "with height", elite[1])
        
    return G

#-----------------[A-D SPECIFIC]------------------#

def initialize(gen_size, n):
    
    global L, D, P
    
    R.<x>           = PolynomialRing(ZZ)
    phi             = x^2 - 1
    phi_factors     = factor(nest(phi,n,x) - x)
    linear_factors  = [phi_factors[i][0]^phi_factors[i][1] for i in range(len(phi_factors))]
    K.<b,c>         = NumberField(linear_factors) 
    L.<a>           = K.galois_closure()
    P.<X,Y>         = ProjectiveSpace(L,1)
    D               = DynamicalSystem([X^2 - Y^2, Y^2])
    generation      = []
    
    while len(generation) < gen_size:
        
        alpha = L.random_element(den_bound = 1)
        
        if not P(alpha).is_preperiodic(D):
            
            generation.append([alpha, D.canonical_height(P(alpha))])
        
    for i in range(len(generation)):
        
        generation[i][0] = toList(generation[i][0])
            
    return generation

def toList(alpha):
    
    alpha_as_list = []
    
    for i in range(alpha.parent().degree()):
        
        alpha_as_list.append(alpha[i])
    
    return alpha_as_list

def toPoint(alpha, L):
    
    alpha_as_point = 0
    
    for i in range(L.degree()):
        
        alpha_as_point += alpha[i]*L.gen()^i
        
    return alpha_as_point

def gradeIndividual(alpha):
    
    alpha = toPoint(alpha, L)
    
    return D.canonical_height(P(alpha))

def isPreperiodic(alpha):
    
    alpha_as_point = toPoint(alpha, L)
    
    if P(alpha_as_point).is_preperiodic(D):
        
        return True
    
    else:
        
        return False
    
#=================================================#
#                    EXECUTION                    #
#=================================================#

#------------------[PARAMETERS]-------------------#

G                = initialize(20,3)
parent_count     = 2
cross_prob       = 0.8
mutate_prob      = 0.1
mutate_space     = [i for i in range(-10,11)]
gradeFunc        = gradeIndividual
generation_count = 100
score_scheme     = "min"
display          = True

#------------------[EXECUTION]--------------------#

geneticAlgorithm(G, parent_count, cross_prob, mutate_prob, mutate_space, gradeFunc, generation_count,score_scheme, display)
            
        